In [1]:
#Import dependencies
import pandas as pd
from config import password
from sqlalchemy import create_engine

In [2]:
#Import nutrition CSV file to pandas dataframes
nutrition_df=pd.read_csv('C:/Users/gjcot/Desktop/Documents/Penn Bootcamp/13_ETL/nutrition.csv')
nutrition_df.head()

,name,Manufacturer,type,calories per serving,grams of protein,grams of fat,milligrams of sodium,grams of dietary fiber,grams of complex carbohydrates,grams of sugars,milligrams of potassium,vitamins and minerals (% of FDA recommendation),Display shelf,Weight in ounces per one serving,Number of cups in one serving,Rating of cereal
0,Apple Cinnamon Cheerios,General Mills,Cold,110,2,2,180,1.5,10.5,10,70,25,1,1.00,0.75,29.509541
1,Basic 4,General Mills,Cold,130,3,2,210,2.0,18.0,8,100,25,3,1.33,0.75,37.038562
2,Cheerios,General Mills,Cold,110,6,2,290,2.0,17.0,1,105,25,1,1.00,1.25,50.764999
3,Cinnamon Toast Crunch,General Mills,Cold,120,1,3,210,0.0,13.0,9,45,25,2,1.00,0.75,19.823573
4,Clusters,General Mills,Cold,110,3,2,140,2.0,13.0,7,105,25,3,1.00,0.50,40.400208


In [3]:
#Rename and clean up values to better join the two dataframes
nutrition_df=nutrition_df.rename(columns={"name":"cereal","Manufacturer":"mfg","type":"type","calories per serving":"calories","grams of protein":"protein_g","grams of fat":"fat_g","milligrams of sodium":"sodium_mg","grams of dietary fiber":"fiber_g","grams of complex carbohydrates":"carbs_g","grams of sugars":"sugar_g","milligrams of potassium":"potassium_mg","vitamins and minerals (% of FDA recommendation)":"vit_min","Display shelf":"shelf","Weight in ounces per one serving":"serving_weight_oz","Number of cups in one serving":"serving_cups","Rating of cereal":"rating"}).copy()

def kelloggs(brand):
    if brand=="Kelloggs":
        return"Kellogg's"
    else:
        return brand
nutrition_df["mfg"]=nutrition_df["mfg"].apply(kelloggs)

nutrition_df["cereal"]=nutrition_df["cereal"].replace("Frosted Mini-Wheats","Frosted Mini Wheats")
nutrition_df["mfg_cereal"]=nutrition_df["mfg"]+" "+nutrition_df["cereal"]
nutrition_df.head()

,cereal,mfg,type,calories,protein_g,fat_g,sodium_mg,fiber_g,carbs_g,sugar_g,potassium_mg,vit_min,shelf,serving_weight_oz,serving_cups,rating,mfg_cereal
0,Apple Cinnamon Cheerios,General Mills,Cold,110,2,2,180,1.5,10.5,10,70,25,1,1.00,0.75,29.509541,General Mills Apple Cinnamon Cheerios
1,Basic 4,General Mills,Cold,130,3,2,210,2.0,18.0,8,100,25,3,1.33,0.75,37.038562,General Mills Basic 4
2,Cheerios,General Mills,Cold,110,6,2,290,2.0,17.0,1,105,25,1,1.00,1.25,50.764999,General Mills Cheerios
3,Cinnamon Toast Crunch,General Mills,Cold,120,1,3,210,0.0,13.0,9,45,25,2,1.00,0.75,19.823573,General Mills Cinnamon Toast Crunch
4,Clusters,General Mills,Cold,110,3,2,140,2.0,13.0,7,105,25,3,1.00,0.50,40.400208,General Mills Clusters


In [4]:
#Read in the second dataframe on market shares
market_share_df=pd.read_csv('C:/Users/gjcot/Desktop/Documents/Penn Bootcamp/13_ETL/market_share.csv')
market_share_df=market_share_df.iloc[1:,:]
market_share_df

,cereal_name,market_share
1,General Mills Honey Nut Cheerios,5.7
2,Post Honey Bunches of Oats,4.7
3,Kellogg's Frosted Flakes,4.6
4,General Mills Cinnamon Toast Crunch,4.1
5,Kellogg's Froot Loops,2.9
6,Kellogg's Frosted Mini Wheats,2.9
7,General Mills Cheerios,2.9
8,General Mills Lucky Charms,2.8
9,Kellogg's Raisin Bran,2.2


In [5]:
# Run install pip, if your computer does not have it installed in jupyter notebook
# Skip this step if it is included in your environment already
!pip install psycopg2

# Set up your connection to Postgres
# Make sure to add your password into the config.py file so that "password" will run
# Otherwise, type your Postgres password directly into the spot for password
rds_connection_string =(f"postgres:{password}@localhost:5432/cereal_db")
engine = create_engine(f"postgresql://{rds_connection_string}")

In [6]:
# Checking that the tables were created correctly in Postgres
engine.table_names()

['nutrition', 'market_share']

In [7]:
# Sending dataframes to Postgres

nutrition_df.to_sql(name='nutrition', con=engine, if_exists='append', index=False)

In [8]:
market_share_df.to_sql(name='market_share', con=engine, if_exists='append', index=False)

In [9]:
#Reading our combined table from Postgres
pd.read_sql_query('select * from nutrition', con=engine).head()

,cereal,mfg,mfg_cereal,type,calories,protein_g,fat_g,sodium_mg,fiber_g,carbs_g,sugar_g,potassium_mg,vit_min,shelf,serving_weight_oz,serving_cups,rating
0,Apple Cinnamon Cheerios,General Mills,General Mills Apple Cinnamon Cheerios,Cold,110,2,2,180,2,11,10,70,25,1,1,1,30
1,Basic 4,General Mills,General Mills Basic 4,Cold,130,3,2,210,2,18,8,100,25,3,1,1,37
2,Cheerios,General Mills,General Mills Cheerios,Cold,110,6,2,290,2,17,1,105,25,1,1,1,51
3,Cinnamon Toast Crunch,General Mills,General Mills Cinnamon Toast Crunch,Cold,120,1,3,210,0,13,9,45,25,2,1,1,20
4,Clusters,General Mills,General Mills Clusters,Cold,110,3,2,140,2,13,7,105,25,3,1,1,40


In [10]:
# Showing our joined table
pd.read_sql_query('select * from market_share', con=engine).head()

,cereal_name,market_share
0,General Mills Honey Nut Cheerios,6
1,Post Honey Bunches of Oats,5
2,Kellogg's Frosted Flakes,5
3,General Mills Cinnamon Toast Crunch,4
4,Kellogg's Froot Loops,3


In [11]:
# Showing our combined dataframes together
pd.read_sql_query('select * from nutrition,market_share where nutrition.mfg_cereal=market_share.cereal_name', con=engine)

,cereal,mfg,mfg_cereal,type,calories,protein_g,fat_g,sodium_mg,fiber_g,carbs_g,sugar_g,potassium_mg,vit_min,shelf,serving_weight_oz,serving_cups,rating,cereal_name,market_share
0,Honey Nut Cheerios,General Mills,General Mills Honey Nut Cheerios,Cold,110,3,1,250,2,12,10,90,25,1,1,1,31,General Mills Honey Nut Cheerios,6
1,Frosted Flakes,Kellogg's,Kellogg's Frosted Flakes,Cold,110,1,0,200,1,14,11,25,25,1,1,1,31,Kellogg's Frosted Flakes,5
2,Cinnamon Toast Crunch,General Mills,General Mills Cinnamon Toast Crunch,Cold,120,1,3,210,0,13,9,45,25,2,1,1,20,General Mills Cinnamon Toast Crunch,4
3,Froot Loops,Kellogg's,Kellogg's Froot Loops,Cold,110,2,1,125,1,11,13,30,25,2,1,1,32,Kellogg's Froot Loops,3
4,Frosted Mini Wheats,Kellogg's,Kellogg's Frosted Mini Wheats,Cold,100,3,0,0,3,14,7,100,25,2,1,1,58,Kellogg's Frosted Mini Wheats,3
5,Cheerios,General Mills,General Mills Cheerios,Cold,110,6,2,290,2,17,1,105,25,1,1,1,51,General Mills Cheerios,3
6,Lucky Charms,General Mills,General Mills Lucky Charms,Cold,110,2,1,180,0,12,12,55,25,2,1,1,27,General Mills Lucky Charms,3
7,Raisin Bran,Kellogg's,Kellogg's Raisin Bran,Cold,120,3,1,210,5,14,12,240,25,2,1,1,39,Kellogg's Raisin Bran,2
